# Random Forest Classifier
In this notebook, we will be using the Decision Trees in order to try and predict individual games. We will be using the data that was collected from the sportsreference API from the summer of 2020. Models will be trained on the moving averages that we generated.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from tqdm import tqdm
from sklearn.model_selection import train_test_split

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Getting Data Ready

In [2]:
# Lead data.csv
data_ma = pd.read_csv('../assets/data/cleaned_data/cleaned_data.csv')
data_ma.head()

,date,location,home,away,winner,home_assist_percentage_SMA,home_assist_percentage_CMA,home_assist_percentage_EMA,home_assists_SMA,home_assists_CMA,home_assists_EMA,home_block_percentage_SMA,home_block_percentage_CMA,home_block_percentage_EMA,home_blocks_SMA,home_blocks_CMA,home_blocks_EMA,home_defensive_rating_SMA,home_defensive_rating_CMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_SMA,home_defensive_rebound_percentage_CMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_SMA,home_defensive_rebounds_CMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_SMA,home_effective_field_goal_percentage_CMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_SMA,home_field_goal_attempts_CMA,home_field_goal_attempts_EMA,home_field_goal_percentage_SMA,home_field_goal_percentage_CMA,home_field_goal_percentage_EMA,home_field_goals_SMA,home_field_goals_CMA,home_field_goals_EMA,home_free_throw_attempt_rate_SMA,home_free_throw_attempt_rate_CMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_SMA,home_free_throw_attempts_CMA,home_free_throw_attempts_EMA,home_free_throw_percentage_SMA,home_free_throw_percentage_CMA,home_free_throw_percentage_EMA,home_free_throws_SMA,home_free_throws_CMA,home_free_throws_EMA,home_losses_SMA,home_losses_CMA,home_losses_EMA,home_minutes_played_SMA,home_minutes_played_CMA,home_minutes_played_EMA,home_offensive_rating_SMA,home_offensive_rating_CMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_SMA,home_offensive_rebound_percentage_CMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_SMA,home_offensive_rebounds_CMA,home_offensive_rebounds_EMA,home_personal_fouls_SMA,home_personal_fouls_CMA,home_personal_fouls_EMA,home_points_SMA,home_points_CMA,home_points_EMA,home_steal_percentage_SMA,home_steal_percentage_CMA,home_steal_percentage_EMA,home_steals_SMA,home_steals_CMA,home_steals_EMA,home_three_point_attempt_rate_SMA,home_three_point_attempt_rate_CMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_SMA,home_three_point_field_goal_attempts_CMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_SMA,home_three_point_field_goal_percentage_CMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_SMA,home_three_point_field_goals_CMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_SMA,home_total_rebound_percentage_CMA,home_total_rebound_percentage_EMA,home_total_rebounds_SMA,home_total_rebounds_CMA,home_total_rebounds_EMA,home_true_shooting_percentage_SMA,home_true_shooting_percentage_CMA,home_true_shooting_percentage_EMA,home_turnover_percentage_SMA,home_turnover_percentage_CMA,home_turnover_percentage_EMA,home_turnovers_SMA,home_turnovers_CMA,home_turnovers_EMA,home_two_point_field_goal_attempts_SMA,home_two_point_field_goal_attempts_CMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_SMA,home_two_point_field_goal_percentage_CMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_SMA,home_two_point_field_goals_CMA,home_two_point_field_goals_EMA,home_win_percentage_SMA,home_win_percentage_CMA,home_win_percentage_EMA,home_wins_SMA,home_wins_CMA,home_wins_EMA,away_assist_percentage_SMA,away_assist_percentage_CMA,away_assist_percentage_EMA,away_assists_SMA,away_assists_CMA,away_assists_EMA,away_block_percentage_SMA,away_block_percentage_CMA,away_block_percentage_EMA,away_blocks_SMA,away_blocks_CMA,away_blocks_EMA,away_defensive_rating_SMA,away_defensive_rating_CMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_SMA,away_defensive_rebound_percentage_CMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_SMA,away_defensive_rebounds_CMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_SMA,away_effective_field_goal_percentage_CMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_SMA,away_field_goal_attempts_CMA,away_field_goal_attempts_EMA,away_field_goal_perc

In [3]:
# These fields aren't necessary to train the model
DROPPING = ['date',
            'location',
            'home',
            'away']

ma = data_ma.copy()
ma['target'] = np.where(ma['winner'] == "Home", 0, 1)
ma.drop(columns=DROPPING, inplace=True)
ma.dropna(inplace=True)
ma.drop_duplicates(inplace=True)
ma.pop('winner')

target = ma.pop('target')
ma = ma.filter(regex="EMA", axis=1)
ma['target'] = target
ma.head()

,home_assist_percentage_EMA,home_assists_EMA,home_block_percentage_EMA,home_blocks_EMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_EMA,home_field_goal_percentage_EMA,home_field_goals_EMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_EMA,home_free_throw_percentage_EMA,home_free_throws_EMA,home_losses_EMA,home_minutes_played_EMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_EMA,home_personal_fouls_EMA,home_points_EMA,home_steal_percentage_EMA,home_steals_EMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_EMA,home_total_rebounds_EMA,home_true_shooting_percentage_EMA,home_turnover_percentage_EMA,home_turnovers_EMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_EMA,home_win_percentage_EMA,home_wins_EMA,away_assist_percentage_EMA,away_assists_EMA,away_block_percentage_EMA,away_blocks_EMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_EMA,away_field_goal_percentage_EMA,away_field_goals_EMA,away_free_throw_attempt_rate_EMA,away_free_throw_attempts_EMA,away_free_throw_percentage_EMA,away_free_throws_EMA,away_losses_EMA,away_minutes_played_EMA,away_offensive_rating_EMA,away_offensive_rebound_percentage_EMA,away_offensive_rebounds_EMA,away_personal_fouls_EMA,away_points_EMA,away_steal_percentage_EMA,away_steals_EMA,away_three_point_attempt_rate_EMA,away_three_point_field_goal_attempts_EMA,away_three_point_field_goal_percentage_EMA,away_three_point_field_goals_EMA,away_total_rebound_percentage_EMA,away_total_rebounds_EMA,away_true_shooting_percentage_EMA,away_turnover_percentage_EMA,away_turnovers_EMA,away_two_point_field_goal_attempts_EMA,away_two_point_field_goal_percentage_EMA,away_two_point_field_goals_EMA,away_win_percentage_EMA,away_wins_EMA,target
0,60.748148,13.012346,4.704938,1.580247,90.790123,82.006173,26.987654,0.477049,53.641975,0.398296,21.555556,0.370296,19.555556,0.800123,15.148148,1.000000,199.802469,102.702469,23.656790,7.518519,13.407407,66.703704,7.507407,4.740741,0.493753,26.271605,0.318815,8.444444,53.501235,34.506173,0.527469,12.735802,9.185185,27.370370,0.463333,13.111111,0.581531,2.395062,49.120988,11.617284,13.429630,5.074074,86.544444,74.550617,25.592593,0.485864,58.802469,0.406642,23.370370,0.397914,21.740741,0.699975,15.370370,1.802469,204.938272,99.222222,28.197531,10.679012,13.481481,70.851852,8.940741,6.444444,0.451827,26.234568,0.343704,8.740741,52.322222,36.271605,0.522778,17.501235,14.148148,32.567901,0.460296,14.629630,0.360444,1.592593,1
1,47.316461,11.905350,9.581481,2.600823,86.714403,87.358436,26.843621,0.524333,54.086420,0.472276,25.654321,0.355288,19.345679,0.642840,12.074074,1.572016,200.000000,108.503292,32.730864,9.333333,15.477366,69.045267,10.812757,6.884774,0.286979,15.687243,0.364667,5.662551,60.924280,36.176955,0.543510,11.871193,8.460905,38.399177,0.522975,19.991770,0.660226,2.691358,38.283951,9.259259,7.653086,2.617284,100.492593,69.771605,24.814815,0.425185,66.000000,0.366716,24.098765,0.323642,21.086420,0.697506,14.827160,2.691358,200.000000,87.290123,21.643210,8.950617,22.987654,70.666667,9.046914,7.407407,0.468247,31.111111,0.255556,7.641975,43.813580,33.765432,0.465407,16.412346,14.716049,34.888889,0.472333,16.456790,0.171556,0.703704,0
2,58.086420,13.345679,8.462963,3.469136,96.674074,76.877778,24.938272,0.502827,53.950617,0.438494,22.740741,0.438173,20.567901,0.717667,14.790123,1.888889,203.703704,88.416049,24.940741,8.395062,17.901235,67.098765,8.839506,6.617284,0.388099,21.333333,0.335617,6.827160,51.477778,33.333333,0.536617,24.403704,20.345679,32.617284,0.502358,15.913580,0.392642,1.506173,55.643621,17.403292,8.8

In [4]:
# create training and test samples
train_ma, test_ma = train_test_split(ma, test_size=0.2)
print(len(train_ma), 'train examples')
print(len(test_ma), 'test examples')

# extract training and test labels
train_true = train_ma.pop('target')
test_true = test_ma.pop('target')

2810 train examples
703 test examples


# Random Forest Classifier

In [125]:
parameters = {'verbose':0,
              'n_jobs':-1,
              'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}
clf = RandomForestClassifier(**parameters)
clf.fit(train_ma, train_true)
accuracy = clf.score(test_ma, test_true)
print(f'Accuracy of model: {accuracy*100:.2f}%')

Accuracy of model: 70.13%


# Gradient Boosting Classifier

In [137]:
parameters = {'verbose':0,
              'loss': 'exponential',
              'n_estimators':100,
              'min_samples_leaf': 3,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 20}
clf = GradientBoostingClassifier(**parameters)
clf.fit(train_ma, train_true)
accuracy = clf.score(test_ma, test_true)
print(f'Accuracy of model: {accuracy*100:.2f}%')

Accuracy of model: 70.55%


# Analysis
As seen above, we were able to surpass 70%, which is quite impressive when predicting sports models. I believe we will be able to increase accuracy when we retrain these models on a larger dataset. Stay tuned when we retrain on our 2010-2020 dataset versus our current 2020 dataset.